In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt

import pathlib
import os
import cv2
import zipfile
from os import path
import shutil
import csv
import random
import numpy as np

# The next few cells can be used to download and train on LPCV Data Directly
# You must upload all LPCV M4V and CSV files to create the dataset

In [ ]:
%mkdir LPCV
%cd LPCV
%mkdir images
%mkdir labels
%mkdir test
%mkdir valid/images
%mkdir valid/labels
%mkdir train
%mkdir train/images
%mkdir train/labels
%cd ..

In [90]:
def capture_frames(video, variety):
  vidcap = cv2.VideoCapture(video)
  success,image = vidcap.read()
  count = 0
  while success:
    cv2.imwrite("LPCV/images/{}_frame{}.jpg".format(variety,count), image)     # save frame as JPEG file      
    success,image = vidcap.read()
    count += 1
  print("Finished Reading " + video)

In [97]:
def create_labels(video_csv, variety):
  with open(video_csv, newline='') as csvfile:
    csv_reader = csv.reader(csvfile)
    yolo_labels = []
    for row in csv_reader:
      frame = row[0]
      if row[0] == "Frame":
        continue
      else:
        with open("LPCV/labels/"+variety+"_frame" + str(frame) + ".txt", "a") as yololabelfile:
          yoloarray = row
          del yoloarray[2:3]
          del yoloarray[0:1]
          for item in yoloarray:
            yololabelfile.write("%s " % item)
          yololabelfile.write("\n")

In [ ]:
capture_frames('4p1b_01A2.m4v','4p1b')
capture_frames('5p2b_01A1.m4v','5p2b')
capture_frames('5p4b_01A2.m4v','5p4b')
capture_frames('5p5b_03A1.m4v','5p5b')
capture_frames('7p3b_02M.m4v','7p3b')

In [98]:
create_labels('4p1b_01A2.csv','4p1b')
create_labels('5p2b_01A1.csv','5p2b')
create_labels('5p4b_01A2.csv','5p4b')
create_labels('5p5b_03A1.csv','5p5b')
create_labels('7p3b_02M.csv','7p3b')

In [48]:
from tensorflow import keras
import tensorflow_datasets as tfds
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import os
import tensorflow as tf
import numpy as np
import glob
from shutil import copyfile
from PIL import Image

In [70]:
%rm -r LPCV/train/labels
%mkdir LPCV/train/labels
%rm -r LPCV/train/images
%mkdir LPCV/train/images

In [102]:
for root, dirs, files in os.walk('LPCV/labels/', topdown=False):
    for filename in files:
      num = filename[10:filename.find(".")]
      name = filename[:10]
      video_file = name+num+".jpg"
      copyfile('LPCV/labels/'+filename, 'LPCV/train/labels/'+filename)
      copyfile('LPCV/images/'+video_file, 'LPCV/train/images/'+video_file)

#To save LPCV image and labels, zip and download the LPCV folder at this point. In the future the folder can be uploaded and the above cells are uneccessary

In [63]:
def contrast_image(image, new_image, folder):
    img = Image.open(folder+'/images/'+image)
    img = np.array(img)
    img = tf.image.random_contrast(img, 2, 5, 42)
    img = img.numpy()
    im = Image.fromarray(img)
    im.save(folder+'/images/'+new_image)
    copy_label(image, "contrast", folder)
def noise_image(image, new_image, folder):
    img = Image.open(folder+'/images/'+image)
    img = np.array(img)
    img = tf.image.stateless_random_jpeg_quality(img, 2, 20, (42,42))
    img = img.numpy()
    im = Image.fromarray(img)
    im.save(folder+'/images/'+new_image)
    copy_label(image, "noise", folder)
def grayscale_image(image, new_image, folder):
    img = Image.open(folder+'/images/'+image)
    img = np.array(img)
    img = tf.image.rgb_to_grayscale(img)
    img = img.numpy()
    img = img.reshape((img.shape[0], img.shape[1]))
    im = Image.fromarray(img, 'L')
    im.save(folder+'/images/'+new_image)
    copy_label(image, "grayscale", folder)
def flip_image_lr(image, new_image, folder):
    img = Image.open(folder+'/images/'+image)
    img = np.array(img)
    img = tf.image.flip_left_right(img)
    img = img.numpy()
    im = Image.fromarray(img)
    im.save(folder+'/images/'+new_image)
    flip_label_lr(image[:image.find('.')]+".txt", image[:image.find('.')]+"_flip_lr.txt", folder)
def flip_image_ud(image, new_image, folder):
    img = Image.open(folder+'/images/'+image)
    img = np.array(img)
    img = tf.image.flip_up_down(img)
    img = img.numpy()
    im = Image.fromarray(img)
    im.save(folder+'/images/'+new_image)
    flip_label_ud(image[:image.find('.')]+".txt", image[:image.find('.')]+"_flip_ud.txt", folder)
def flip_label_lr(label_file, new_label_file, folder):
    label_file = folder+"/labels/"+label_file
    new_label_file = folder+"/labels/"+new_label_file
    with open(label_file) as f:
        lines = f.readlines()
        new_lines = []
        for x in lines:
            words = x.split(" ")
            words[1] = str(1-float(words[1]))
            new_lines.append(" ".join(words))
    f.close()
    with open(new_label_file, 'w') as f:
        for x in new_lines:
            f.write(x)
def flip_label_ud(label_file, new_label_file, folder):
    label_file = folder+"/labels/"+label_file
    new_label_file = folder+"/labels/"+new_label_file
    with open(label_file) as f:
        lines = f.readlines()
        new_lines = []
        for x in lines:
            words = x.split(" ")
            words[2] = str(1-float(words[2]))
            new_lines.append(" ".join(words))
    f.close()
    with open(new_label_file, 'w') as f:
        for x in new_lines:
            f.write(x)
def copy_label(image, augmentation, folder):
    filename = image[:image.find('.')]
    new_filename = folder+'/labels/'+filename+'_'+augmentation+'.txt'
    filename = folder+'/labels/'+filename+'.txt'
    copyfile(filename, new_filename)

In [72]:
aug_list = ['_contrast', '_noise', '_grayscale', '_flip_lr', '_flip_ud']
data_dir = "./LPCV/train"

In [104]:
for root, dirs, files in os.walk(data_dir+'/images/', topdown=False):
    for file in files:
      count = 0
      for j in aug_list:
        filename = file[:file.find('.')]+j+file[file.find('.'):]
        if count==0: contrast_image(file, filename, data_dir)
        if count==1: noise_image(file,filename, data_dir)
        if count==2: grayscale_image(file, filename, data_dir)
        if count==3: flip_image_lr(file, filename, data_dir)
        if count==4: flip_image_ud(file, filename, data_dir)
        count+=1

In [110]:
def moveFiles(src, dst):
  files = []
  for i in os.listdir(src):
    files.append(i)
  files = sorted(files)
  listt = np.random.RandomState(seed=42).permutation(files)[:30]
  for f in listt:
    shutil.copy(os.path.join(src, f), dst)
    p = os.path.join(src, f)
    os.remove(p)

In [112]:
moveFiles("LPCV/train/images", "LPCV/valid/images")
moveFiles("LPCV/train/labels", "LPCV/valid/labels")

In [113]:
data = open("data.yaml", "w")
data.write("train: ./LPCV/train/images\nval: ./LPCV/valid/images\nnc: 2\nnames: [\'person\', \'sports-ball\']")
data.close()

In [ ]:
!python train.py --img 416 --batch 50 --epochs 10 --data ./data.yaml --weights yolov5s.pt

In [ ]:
 !python detect.py --weights runs/train/exp/weights/best.pt --img 640 --conf 0.05 --source 5p5b_03A1.m4v